## Learning in Bayesian Networks

### Learning Network Conditional Probability Tables

Let's start learning the parameters (CPDs) of a Discrete Bayesian Network given the data and the model structure. pgmpy has three main methods for learning the parameters: MaximumLikelihood Estimator (pgmpy.estimators.MaximumLikelihoodEstimator), Bayesian Estimator (pgmpy.estimators.BayesianEstimator), and Expectation Maximization (pgmpy.estimators.ExpectationMaximization)

#### Data Generation

In the examples, we will try to generate some data from given models and then try to learn the model parameters back from the generated data.

In [1]:
# Use the alarm model to generate data from it.

from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling

alarm_model = get_example_model("alarm")
samples = BayesianModelSampling(alarm_model).forward_sample(size=int(1e4))
samples.head()

  0%|          | 0/37 [00:00<?, ?it/s]

,HISTORY,CVP,PCWP,HYPOVOLEMIA,LVEDVOLUME,LVFAILURE,STROKEVOLUME,ERRLOWOUTPUT,HRBP,HREKG,...,MINVOLSET,VENTMACH,VENTTUBE,VENTLUNG,VENTALV,ARTCO2,CATECHOL,HR,CO,BP
0,FALSE,NORMAL,HIGH,TRUE,HIGH,FALSE,LOW,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,LOW,LOW
1,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,HIGH,HIGH,...,LOW,LOW,ZERO,LOW,HIGH,LOW,HIGH,HIGH,HIGH,HIGH
2,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,LOW,LOW,...,NORMAL,NORMAL,HIGH,LOW,HIGH,LOW,NORMAL,NORMAL,NORMAL,NORMAL
3,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,HIGH,LOW
4,FALSE,HIGH,HIGH,TRUE,HIGH,FALSE,LOW,TRUE,NORMAL,HIGH,...,NORMAL,NORMAL,ZERO,LOW,HIGH,LOW,HIGH,HIGH,NORMAL,LOW


In this case, since we are trying to learn the model parameters back we will use the model structure that we used to generate the data from.

In [2]:
# Defining the Bayesian Model structure

from pgmpy.models import BayesianNetwork

model_struct = BayesianNetwork(ebunch=alarm_model.edges())
print(model_struct.nodes())
print()
print(model_struct.edges())


['HYPOVOLEMIA', 'LVEDVOLUME', 'STROKEVOLUME', 'CVP', 'PCWP', 'LVFAILURE', 'HISTORY', 'CO', 'ERRLOWOUTPUT', 'HRBP', 'ERRCAUTER', 'HREKG', 'HRSAT', 'INSUFFANESTH', 'CATECHOL', 'ANAPHYLAXIS', 'TPR', 'BP', 'KINKEDTUBE', 'PRESS', 'VENTLUNG', 'FIO2', 'PVSAT', 'SAO2', 'PULMEMBOLUS', 'PAP', 'SHUNT', 'INTUBATION', 'MINVOL', 'VENTALV', 'DISCONNECT', 'VENTTUBE', 'MINVOLSET', 'VENTMACH', 'EXPCO2', 'ARTCO2', 'HR']

[('HYPOVOLEMIA', 'LVEDVOLUME'), ('HYPOVOLEMIA', 'STROKEVOLUME'), ('LVEDVOLUME', 'CVP'), ('LVEDVOLUME', 'PCWP'), ('STROKEVOLUME', 'CO'), ('LVFAILURE', 'HISTORY'), ('LVFAILURE', 'LVEDVOLUME'), ('LVFAILURE', 'STROKEVOLUME'), ('CO', 'BP'), ('ERRLOWOUTPUT', 'HRBP'), ('ERRCAUTER', 'HREKG'), ('ERRCAUTER', 'HRSAT'), ('INSUFFANESTH', 'CATECHOL'), ('CATECHOL', 'HR'), ('ANAPHYLAXIS', 'TPR'), ('TPR', 'CATECHOL'), ('TPR', 'BP'), ('KINKEDTUBE', 'PRESS'), ('KINKEDTUBE', 'VENTLUNG'), ('VENTLUNG', 'EXPCO2'), ('VENTLUNG', 'MINVOL'), ('VENTLUNG', 'VENTALV'), ('FIO2', 'PVSAT'), ('PVSAT', 'SAO2'), ('SAO2', '

In [3]:
# Fitting the model using Maximum Likelihood Estimator

from pgmpy.estimators import MaximumLikelihoodEstimator

mle = MaximumLikelihoodEstimator(model=model_struct, data=samples)

# Estimating the CPD for a single node.
print(mle.estimate_cpd(node="FIO2"))
print()
print(mle.estimate_cpd(node="CVP"))

# Estimating CPDs for all the nodes in the model
mle.get_parameters()[:10]  # Show just the first 10 CPDs in the output

+--------------+--------+
| FIO2(LOW)    | 0.0482 |
+--------------+--------+
| FIO2(NORMAL) | 0.9518 |
+--------------+--------+

+-------------+-----+----------------------+
| LVEDVOLUME  | ... | LVEDVOLUME(NORMAL)   |
+-------------+-----+----------------------+
| CVP(HIGH)   | ... | 0.008892025405786874 |
+-------------+-----+----------------------+
| CVP(LOW)    | ... | 0.043048694424841216 |
+-------------+-----+----------------------+
| CVP(NORMAL) | ... | 0.9480592801693719   |
+-------------+-----+----------------------+


[<TabularCPD representing P(HYPOVOLEMIA:2) at 0x7f4ca565ad70>,
 <TabularCPD representing P(LVEDVOLUME:3 | HYPOVOLEMIA:2, LVFAILURE:2) at 0x7f4ca565b040>,
 <TabularCPD representing P(STROKEVOLUME:3 | HYPOVOLEMIA:2, LVFAILURE:2) at 0x7f4ca565b310>,
 <TabularCPD representing P(CVP:3 | LVEDVOLUME:3) at 0x7f4ca565b610>,
 <TabularCPD representing P(PCWP:3 | LVEDVOLUME:3) at 0x7f4ca565b4f0>,
 <TabularCPD representing P(LVFAILURE:2) at 0x7f4ca565be50>,
 <TabularCPD representing P(HISTORY:2 | LVFAILURE:2) at 0x7f4ca569a230>,
 <TabularCPD representing P(CO:3 | HR:3, STROKEVOLUME:3) at 0x7f4ca5659690>,
 <TabularCPD representing P(ERRLOWOUTPUT:2) at 0x7f4ca5698ca0>,
 <TabularCPD representing P(HRBP:3 | ERRLOWOUTPUT:2, HR:3) at 0x7f4ca56992d0>]

In [4]:
import numpy as np

# Verifying that the learned parameters are almost equal.
print(
    np.allclose(
    alarm_model.get_cpds("FIO2").values, mle.estimate_cpd("FIO2").values, atol=0.01
))
print()
print(alarm_model.get_cpds("FIO2"))
print()
print(mle.estimate_cpd("FIO2"))

True

+--------------+------+
| FIO2(LOW)    | 0.05 |
+--------------+------+
| FIO2(NORMAL) | 0.95 |
+--------------+------+

+--------------+--------+
| FIO2(LOW)    | 0.0482 |
+--------------+--------+
| FIO2(NORMAL) | 0.9518 |
+--------------+--------+


In [9]:
# Fitting the using Bayesian Estimator
from pgmpy.estimators import BayesianEstimator

best = BayesianEstimator(model=model_struct, data=samples)

print(best.estimate_cpd(node="FIO2", prior_type="BDeu", equivalent_sample_size=100))
# Uniform pseudo count for each state. Can also accept an array of the size of CPD.
print(best.estimate_cpd(node="CVP", prior_type="dirichlet", pseudo_counts=100))

# Learning CPDs for all the nodes in the model. For learning all parameters with BDeU prior, a dict of
# pseudo_counts need to be provided
best.get_parameters(prior_type="BDeu", equivalent_sample_size=1000)[:10]

+--------------+-----------+
| FIO2(LOW)    | 0.0526733 |
+--------------+-----------+
| FIO2(NORMAL) | 0.947327  |
+--------------+-----------+
+-------------+---------------------+-----+---------------------+
| LVEDVOLUME  | LVEDVOLUME(HIGH)    | ... | LVEDVOLUME(NORMAL)  |
+-------------+---------------------+-----+---------------------+
| CVP(HIGH)   | 0.6609315988250105  | ... | 0.02207176709546378 |
+-------------+---------------------+-----+---------------------+
| CVP(LOW)    | 0.04951741502308015 | ... | 0.05484089370345294 |
+-------------+---------------------+-----+---------------------+
| CVP(NORMAL) | 0.2895509861519094  | ... | 0.9230873392010833  |
+-------------+---------------------+-----+---------------------+


[<TabularCPD representing P(HYPOVOLEMIA:2) at 0x7f4ca45c7fa0>,
 <TabularCPD representing P(LVEDVOLUME:3 | HYPOVOLEMIA:2, LVFAILURE:2) at 0x7f4ca45c6aa0>,
 <TabularCPD representing P(STROKEVOLUME:3 | HYPOVOLEMIA:2, LVFAILURE:2) at 0x7f4ca45c6770>,
 <TabularCPD representing P(CVP:3 | LVEDVOLUME:3) at 0x7f4ca45c5000>,
 <TabularCPD representing P(PCWP:3 | LVEDVOLUME:3) at 0x7f4ca5739c90>,
 <TabularCPD representing P(LVFAILURE:2) at 0x7f4ca45c76a0>,
 <TabularCPD representing P(HISTORY:2 | LVFAILURE:2) at 0x7f4ca45c5ed0>,
 <TabularCPD representing P(CO:3 | HR:3, STROKEVOLUME:3) at 0x7f4ca45c4be0>,
 <TabularCPD representing P(ERRLOWOUTPUT:2) at 0x7f4ca45c68c0>,
 <TabularCPD representing P(HRBP:3 | ERRLOWOUTPUT:2, HR:3) at 0x7f4ca45c5ff0>]

In [10]:
# Shortcut for learning all the parameters and adding the CPDs to the model.

model_struct = BayesianNetwork(ebunch=alarm_model.edges())
model_struct.fit(data=samples, estimator=MaximumLikelihoodEstimator)
print(model_struct.get_cpds("FIO2"))

model_struct = BayesianNetwork(ebunch=alarm_model.edges())
model_struct.fit(
    data=samples,
    estimator=BayesianEstimator,
    prior_type="BDeu",
    equivalent_sample_size=1000,
)
print(model_struct.get_cpds("FIO2"))

+--------------+--------+
| FIO2(LOW)    | 0.0482 |
+--------------+--------+
| FIO2(NORMAL) | 0.9518 |
+--------------+--------+
+--------------+-----------+
| FIO2(LOW)    | 0.0892727 |
+--------------+-----------+
| FIO2(NORMAL) | 0.910727  |
+--------------+-----------+


In [11]:
from pgmpy.estimators import ExpectationMaximization as EM

# Define a model structure with latent variables
model_latent = BayesianNetwork(
    ebunch=alarm_model.edges(), latents=["HYPOVOLEMIA", "LVEDVOLUME", "STROKEVOLUME"]
)

# Dataset for latent model which doesn't have values for the latent variables
samples_latent = samples.drop(model_latent.latents, axis=1)

model_latent.fit(samples_latent, estimator=EM)

  0%|          | 0/100 [00:00<?, ?it/s]

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_count_data = data.groupby([variable]).sum()["_weight"]
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data.groupby([variable] + parents).sum()["_weight"].unstack(parents)
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, num

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data.groupby([variable] + parents).sum()["_weight"].unstack(parents)
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_count_data = data.groupby([variable]).sum()["_weight"]
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, num

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_count_data = data.groupby([variable]).sum()["_weight"]
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data.groupby([variable] + parents).sum()["_weight"].unstack(parents)
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, num

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_count_data = data.groupby([variable]).sum()["_weight"]
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_count_data = data.groupby([variable]).sum()["_weight"]
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_onl

...

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_count_data = data.groupby([variable]).sum()["_weight"]
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_count_data = data.groupby([variable]).sum()["_weight"]
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_onl

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_count_data = data.groupby([variable]).sum()["_weight"]
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_count_data = data.groupby([variable]).sum()["_weight"]
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_onl

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data.groupby([variable] + parents).sum()["_weight"].unstack(parents)
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_count_data = data.groupby([variable]).sum()["_weight"]
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, num

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_count_data = data.groupby([variable]).sum()["_weight"]
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:141: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  state_count_data = data.groupby([variable]).sum()["_weight"]
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_onl

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data.groupby([variable] + parents).sum()["_weight"].unstack(parents)
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/base.py:156: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data.groupby([variable] + parents).sum()["_weight"].unstack(parents)


In [12]:
print(alarm_model.get_cpds("FIO2"))
print()
print(mle.estimate_cpd("FIO2"))
print()
print(model_struct.get_cpds("FIO2"))
print()
print(model_latent.get_cpds("FIO2"))

+--------------+------+
| FIO2(LOW)    | 0.05 |
+--------------+------+
| FIO2(NORMAL) | 0.95 |
+--------------+------+

+--------------+--------+
| FIO2(LOW)    | 0.0482 |
+--------------+--------+
| FIO2(NORMAL) | 0.9518 |
+--------------+--------+

+--------------+-----------+
| FIO2(LOW)    | 0.0892727 |
+--------------+-----------+
| FIO2(NORMAL) | 0.910727  |
+--------------+-----------+

+--------------+--------+
| FIO2(LOW)    | 0.0482 |
+--------------+--------+
| FIO2(NORMAL) | 0.9518 |
+--------------+--------+


In [13]:
print(alarm_model.get_cpds("LVEDVOLUME"))
print()
print(mle.estimate_cpd("LVEDVOLUME"))
print()
print(model_struct.get_cpds("LVEDVOLUME"))
print()
print(model_latent.get_cpds("LVEDVOLUME"))

+--------------------+-----+--------------------+
| HYPOVOLEMIA        | ... | HYPOVOLEMIA(FALSE) |
+--------------------+-----+--------------------+
| LVFAILURE          | ... | LVFAILURE(FALSE)   |
+--------------------+-----+--------------------+
| LVEDVOLUME(LOW)    | ... | 0.05               |
+--------------------+-----+--------------------+
| LVEDVOLUME(NORMAL) | ... | 0.9                |
+--------------------+-----+--------------------+
| LVEDVOLUME(HIGH)   | ... | 0.05               |
+--------------------+-----+--------------------+

+--------------------+-----+-------------------+
| HYPOVOLEMIA        | ... | HYPOVOLEMIA(TRUE) |
+--------------------+-----+-------------------+
| LVFAILURE          | ... | LVFAILURE(TRUE)   |
+--------------------+-----+-------------------+
| LVEDVOLUME(HIGH)   | ... | 0.0               |
+--------------------+-----+-------------------+
| LVEDVOLUME(LOW)    | ... | 0.9625            |
+--------------------+-----+-------------------+
| LVEDVO

### Learning the network structure

Currently, pgmpy has implementation of 3 main algorithms:
- PC with stable and parallel variants
- Hill-Climb Search 
- Exhaustive Search

For PC the following conditional independence test can be used: 
- Chi-Square test (https://en.wikipedia.org/wiki/Chi-squared_test) 
- Pearsonr (https://en.wikipedia.org/wiki/Partial_correlation#Using_linear_regression) 
- G-squared (https://en.wikipedia.org/wiki/G-test) 
- Log-likelihood (https://en.wikipedia.org/wiki/G-test)
- Freeman-Tuckey (Read, Campbell B. “Freeman—Tukey chi-squared goodness-of-fit statistics.” Statistics & probability letters 18.4 (1993): 271-278.)
- Modified Log-likelihood
- Neymann (https://en.wikipedia.org/wiki/Neyman%E2%80%93Pearson_lemma) 
- Cressie Read (Cressie, Noel, and Timothy RC Read. “Multinomial goodness‐of‐fit tests.” Journal of the Royal Statistical Society: Series B (Methodological) 46.3 (1984): 440-464)
- Power Divergence (Cressie, Noel, and Timothy RC Read. “Multinomial goodness‐of‐fit tests.” Journal of the Royal Statistical Society: Series B (Methodological) 46.3 (1984): 440-464.)

For Hill-Climb and Exhausitive Search the following scoring methods can be used: 
- K2 Score 
- BDeu Score
- Bic Score

#### Generate some data

In [14]:
from itertools import combinations

import networkx as nx
from sklearn.metrics import f1_score

from pgmpy.estimators import PC, HillClimbSearch, ExhaustiveSearch
from pgmpy.estimators import K2Score
from pgmpy.utils import get_example_model
from pgmpy.sampling import BayesianModelSampling

In [15]:
model = get_example_model("alarm")
samples = BayesianModelSampling(model).forward_sample(size=int(1e3))
samples.head()

  0%|          | 0/37 [00:00<?, ?it/s]

,HISTORY,CVP,PCWP,HYPOVOLEMIA,LVEDVOLUME,LVFAILURE,STROKEVOLUME,ERRLOWOUTPUT,HRBP,HREKG,...,MINVOLSET,VENTMACH,VENTTUBE,VENTLUNG,VENTALV,ARTCO2,CATECHOL,HR,CO,BP
0,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,LOW,NORMAL
1,FALSE,NORMAL,HIGH,FALSE,NORMAL,FALSE,NORMAL,FALSE,LOW,LOW,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,NORMAL,NORMAL,LOW
2,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,LOW,ZERO,LOW,HIGH,HIGH,HIGH,HIGH,LOW
3,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,HIGH,HIGH
4,FALSE,NORMAL,NORMAL,FALSE,NORMAL,FALSE,NORMAL,FALSE,HIGH,HIGH,...,NORMAL,NORMAL,LOW,ZERO,ZERO,HIGH,HIGH,HIGH,HIGH,HIGH


In [16]:
# Funtion to evaluate the learned model structures.
def get_f1_score(estimated_model, true_model):
    nodes = estimated_model.nodes()
    est_adj = nx.to_numpy_matrix(
        estimated_model.to_undirected(), nodelist=nodes, weight=None
    )
    true_adj = nx.to_numpy_matrix(
        true_model.to_undirected(), nodelist=nodes, weight=None
    )

    f1 = f1_score(np.ravel(true_adj), np.ravel(est_adj))
    print("F1-score for the model skeleton: ", f1)

#### Learn the model structure using PC

In [17]:
est = PC(data=samples)
estimated_model = est.estimate(variant="stable", max_cond_vars=4)
get_f1_score(estimated_model, model)

  0%|          | 0/4 [00:00<?, ?it/s]

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

...

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

F1-score for the model skeleton:  0.7945205479452055


In [18]:
est = PC(data=samples)
estimated_model = est.estimate(variant="orig", max_cond_vars=4)
get_f1_score(estimated_model, model)

  0%|          | 0/4 [00:00<?, ?it/s]

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

...

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/matteo/anaconda3/envs/bn_env/lib/python3.10/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper

F1-score for the model skeleton:  0.8157894736842105


#### Learn the model structure using Hill-Climb Search

In [19]:
scoring_method = K2Score(data=samples)
est = HillClimbSearch(data=samples)
estimated_model = est.estimate(
    scoring_method=scoring_method, max_indegree=4, max_iter=int(1e4)
)
get_f1_score(estimated_model, model)

  0%|          | 0/10000 [00:00<?, ?it/s]

F1-score for the model skeleton:  0.8200000000000001
